In [ ]:
def single_multiple_choice(question, choices, answer, hint, comment):
    # Make the required widgets
    question_widget = pn.widgets.StaticText(value=question)
    choices_widget = pn.widgets.Select(options=choices, name="")
    submit_button = pn.widgets.Button(name="Check")
    feedback_widget = pn.widgets.TextInput(value="", name="", width=500)

    # Build the structure by aligning the widgets
    submit_row = pn.Row(submit_button, feedback_widget)
    quiz_widget = pn.Column(question_widget, choices_widget, submit_row)
    display(quiz_widget)

    def check_answers(event):
        chosen_answer = choices_widget.value

        if chosen_answer == answer:
            feedback_widget.value = comment
        else:
            feedback_widget.value = hint

    # Run the function check_answers when the submit button is pressed
    submit_button.on_click(check_answers)

In [ ]:
def multiple_selection(question, correct_statements, false_statements):
    # Make an empty list to store the widgets (references), checkboxes, and true/false statements sorted.
    check_boxes = []  # all the boxes to click
    all_statements = []  # all the statements

    # question widget
    question_widget = pn.widgets.StaticText(value=question)

    # For visualization, the maximum length/width of the question, with a minimum width of 200
    max_length = max(len(item) for item in correct_statements + false_statements)
    width_statement = max(250, max_length * 7)

    # An empty list for visualization to store the HBoxes that contain the widgets, one statement and the corresponding checkbox
    all_widgets = []

    for statement in correct_statements + false_statements:
        add_statement = pn.widgets.StaticText(value=statement, width=width_statement)
        check_box_widget = pn.widgets.Checkbox(value=False, width=120)
        HBox1 = pn.Row(add_statement, check_box_widget)

        all_statements.append(add_statement)
        check_boxes.append(check_box_widget)
        all_widgets.append(HBox1)

    # randomize the order of statements
    shuffle(all_widgets)

    # add submit button and output, which come on the bottom
    submit_button = pn.widgets.Button(name="Check")
    output_widget = pn.widgets.TextInput(value="", placeholder="", disabled=False)

    # make an additional HBox for aligning the submit button and the output widget
    HBox2 = pn.Row(submit_button, output_widget)
    all_widgets.append(HBox2)

    # align all the HBoxes beneath each other (oldest below if not randomized) and display them.
    quiz_widget = pn.Column(question_widget, *all_widgets)

    # Check the checkbox for each statement and calculate the score.
    def check_answers(event):
        score = 0

        for i in range(len(check_boxes)):
            check_box = check_boxes[i]
            statement = all_statements[i].value

            if statement in correct_statements:
                if check_box.value == True:
                    score += 1
                else:
                    score -= 0

            if statement not in correct_statements:
                if check_box.value == True:
                    score -= 1
                else:
                    score -= 0

        score = np.max([score, 0])
        output_widget.value = (
            f"You have: {round(score/len(correct_statements)*100,0)}% of the points"
        )

    submit_button.on_click(check_answers)
    return quiz_widget  # This is similar to .serve(), it can also be done through: display(quiz_widget)

In [ ]:
def nummeric_question_body(
    questions, units, answers, FB_good, FB_wrong, random_order=False, f_margin=0
):
    all_widgets = []
    attempts = []

    order = np.arange(0, len(questions), 1)
    if random_order == True:
        shuffle(order)

    for i in np.array(order):
        question, unit, answer, Q_FB_G, Q_FB_W = (
            questions[i],
            units[i],
            answers[i],
            FB_good[i],
            FB_wrong[i],
        )
        id = i + 1
        question_widget = pn.widgets.StaticText(value=question, width=750)
        unit_widget = pn.widgets.StaticText(value=unit, width=40)
        num_widget = pn.widgets.FloatInput(value=0, step=0.01, width=100)
        # feedback_widget = pn.widgets.TextInput(value="", name="", width=500)
        feedback_widget = pn.widgets.StaticText(value="", name="", width=500)
        submit_button = pn.widgets.Button(name="Check")

        Hbox = pn.Row(num_widget, unit_widget, submit_button, feedback_widget)
        quiz_widget = pn.Column(question_widget, Hbox)

        all_widgets.append(quiz_widget)

        # the values for the submit button are determined at the moment these are created.
        attempt = pn.widgets.FloatInput(value=0)
        attempts.append(attempt)
        submit_button.on_click(
            check_nummeric_answers(
                id,
                answer,
                unit,
                Q_FB_G,
                Q_FB_W,
                num_widget,
                feedback_widget,
                attempt,
                f_margin,
            )
        )

    return all_widgets

In [ ]:
def limit_answer(answer, f_margin=0):
    # if no margin is given, adding additional decimals are counted correctly.
    # if a margin if given, the answer should be within a range from the correct answer.
    # The maximal deviation is determined with the factor (f_margin)
    s = str(answer)

    if f_margin == 0:
        # inspired by: https://stackoverflow.com/questions/35585950/find-the-number-of-digits-after-the-decimal-point
        if not "." in s:
            n_decimal = 0
        else:
            n_decimal = len(s) - s.index(".") - 1

        range = 5 * 10 ** -(n_decimal + 1)

    if f_margin != 0:
        range = f_margin * answer

    return range


def check_nummeric_answers(
    id, answer, unit, FB_G, FB_W, num_widget, feedback_widget, attempt, f_margin=0
):
    def button_callback(b):
        attempt.value += 1
        # print("debug question:", id, 'attempt', attempt.value ,', response:' ,num_widget.value, ', answer:',answer)

        # the answer is within the boundaries, print positive feedback
        if np.abs(answer - num_widget.value) < limit_answer(answer, f_margin):
            if len(FB_G) != 0:
                feedback_widget.value = FB_G
            else:
                feedback_widget.value = "Well done, this is correct!"

        # the answer is NOT within boundaries, provide feedback based on the number of attempts
        if np.abs(answer - num_widget.value) >= limit_answer(answer, f_margin):
            if attempt.value < 3 and len(FB_W) > 0:
                feedback_widget.value = FB_W

            if attempt.value < 3 and len(FB_W) == 0:
                feedback_widget.value = "Oops, there seems to be a mistake"

            if attempt.value >= 3:
                feedback_widget.value = (
                    "The correct answer is " + str(answer) + str(unit) + "."
                )

    return (
        button_callback  # otherwise gives TypeError: 'NoneType' object is not callable
    )

In [ ]:
class class_variables:
    def __setattr__(self, key, value):
        object.__setattr__(self, key, value)


def classify_variables(locals, max_size_MB=0):
    max_size = max_size_MB * 1024 * 1024
    FV = class_variables()
    for key, value in {**globals(), **locals}.items():
        if sys.getsizeof(value) < max_size or max_size <= 0:
            FV.__setattr__(key, value)
    return FV


def add_local_variables_to(FV, locals, max_size_MB=0):
    max_size = max_size_MB * 1024 * 1024
    for key, value in locals.items():
        if sys.getsizeof(value) < max_size or max_size <= 0:
            FV.__setattr__(key, value)
    return FV


def check_code_values(IV):
    # The function variable (FV) is now defined as Input Variable (IV) (= arguments)
    # The input variable and the newly defined local variable will be merged to FV

    def get_coded_values(FV, GV):
        def button_callback(b):
            try:
                for i, param in enumerate(FV.check_parameters):
                    # get the value that students gave
                    response = getattr(GV, param)

                    # store them in the widget
                    FV.all_parameter_widgets[i].value = response

                FV.debug_widget.value = ""
            except:
                FV.debug_widget.value = (
                    "<b> Careful, not all parameters are defined! </b>"
                )

        return button_callback

    def check_coded_values(FV, GV):
        def button_callback(b):
            FV.attempt.value += 1

            for i, param in enumerate(FV.check_parameters):
                # response = getattr(GV, param)
                response = FV.all_parameter_widgets[i].value
                answer = getattr(FV, param)

                if np.abs(response - answer) < FV.f_margin * answer:
                    FV.all_feedback_widgets[i].value = "Nice, this is good!"

                if np.abs(response - answer) >= FV.f_margin * answer:
                    if FV.attempt.value < 3:
                        FV.all_feedback_widgets[
                            i
                        ].value = "This one is incorrect, try again!"
                    if FV.attempt.value >= 3:
                        FV.all_feedback_widgets[i].value = (
                            "This one is incorrect, the answer should be "
                            + str(round(answer, FV.n_decimals))
                            + "."
                        )

                # print('Debug: ', param, response, answer)

        return button_callback

    all_parameter_widgets = []
    info_widgets = []
    all_feedback_widgets = []
    for name, param in zip(IV.name_parameters, IV.check_parameters):
        symbol_widget = pn.widgets.StaticText(name="", value=name, width=100)
        parameter_widget = pn.widgets.FloatInput(name="", value=0, width=100)
        feedback_widget = pn.widgets.StaticText(value="")

        all_parameter_widgets.append(parameter_widget)
        all_feedback_widgets.append(feedback_widget)

        new_row = pn.Row(symbol_widget, parameter_widget, feedback_widget)
        info_widgets.append(new_row)

    debug_widget = pn.widgets.StaticText(value="")
    # attempt = pn.widgets.FloatInput(value=0)

    get_values_button = pn.widgets.Button(name="Load values")
    check_values_button = pn.widgets.Button(name="Check loaded values")

    GV = classify_variables(globals())
    FV = add_local_variables_to(IV, locals())

    # The error margin is set at 0.01% if it is not defined
    if "f_margin" not in FV.__dict__:
        FV.__setattr__("f_margin", 0.0001)

    if "n_decimals" not in FV.__dict__:
        FV.__setattr__("n_decimals", 10)

    get_values_button.on_click(get_coded_values(FV, GV))
    check_values_button.on_click(check_coded_values(FV, GV))

    row_buttons = pn.Row(get_values_button, check_values_button, debug_widget)
    Input_col = pn.Column(row_buttons, *info_widgets)
    display(Input_col)

In [ ]:
def check_code_function(
    fig, horizontal_axis, function_name, correct_function, par_x_axis, f_margin=0.005
):
    # The error margin (f_margin) is set at 0.1% if it is not defined

    # make the graph and show the correct answer
    ax = fig.subplots()
    pane = pn.pane.Matplotlib(fig, dpi=100)

    #  Plot the answer if the student function is found
    try:
        # Load the student-made function from globals
        function = globals()[function_name]

        # Add the arguments to the name of the function, for eval()
        # + Replace the argument on the x-axis so that it can be assessed with list comprehension.
        # https://docs.python.org/3/library/inspect.html
        # https://peps.python.org/pep-0362/
        sig = signature(function)
        sig_function = str(sig).replace(par_x_axis, "par_x_axis")
        function = function_name + str(sig_function)
        title = function_name + str(sig)
        title = title.replace("_", " ")

        sig = signature(correct_function)
        sig_function = str(sig).replace(par_x_axis, "par_x_axis")
        correct_function2 = str(correct_function.__name__) + sig_function

        # calculate the student's answer through eval()
        student_answer = [eval(function) for par_x_axis in horizontal_axis]

        # calculate the correct answer
        correct_answer = []
        for par_x_axis in horizontal_axis:
            correct_answer.append(eval(correct_function2))
        # This code should work, but it does not
        # correct_answer = [eval(correct_function2) for par_x_axis in horizontal_axis]

        # check if the answer is correct and plot it before/below lines
        changes = np.array(student_answer) - np.array(correct_answer)
        inaccuracy = np.abs(1 - np.array(correct_answer) / np.array(student_answer))

        if np.max(changes) == 0:
            y_loc = (np.mean(correct_answer) + np.min(correct_answer)) / 2
            text = ax.text(
                np.mean([horizontal_axis]),
                y_loc,
                "Perfect!",
                fontsize=12,
                color="#1b5a00",
                ha="center",
                va="center",
            )

        if np.max(changes) != 0 and np.max(inaccuracy) < f_margin:
            y_loc = (np.mean(correct_answer) + np.min(correct_answer)) / 2
            text = ax.text(
                np.mean([horizontal_axis]),
                y_loc,
                "Good!",
                fontsize=12,
                color="#1b5a00",
                ha="center",
                va="center",
            )

        # plot the answers
        line = ax.plot(horizontal_axis, student_answer, label="Your answer")
        line = ax.plot(horizontal_axis, correct_answer, label="Correct answer")
        ax.legend()
        ax.set_title(title)

    except:
        text_failed = "Almost there, \n your function can not be plotted, \n please try to fix the bug"
        x_ticks = ax.get_xticks()
        y_ticks = ax.get_yticks()
        text = ax.text(
            np.average(x_ticks),
            np.average(y_ticks),
            text_failed,
            fontsize=16,
            color="r",
            ha="center",
            va="center",
        )

    # update the graph
    display(pane)

The code below repplaces the previous version. The function will be rewritten. 

In [ ]:
def check_code_function(
    fig,
    horizontal_axis,
    function_name,
    correct_function,
    par_x_axis,
    f_margin=0.005,
    new_graph=True,
    ax=None,
    pane=None,
    xlabel=None,
    ylabel=None,
):
    if new_graph == True:
        pane = pn.pane.Matplotlib(fig, dpi=100)
        ax = fig.subplots()

    if xlabel != None:
        ax.set_xlabel(xlabel)
        fig.subplots_adjust(bottom=0.25)

    if ylabel != None:
        ax.set_ylabel(ylabel)
        fig.subplots_adjust(left=0.1)

    #  Plot the answer if the student function is found
    try:
        # Load the student-made function from globals
        function = globals()[function_name]

        # Add the arguments to the name of the function, for eval()
        # + Replace the argument on the x-axis so that it can be assessed with list comprehension.
        # https://docs.python.org/3/library/inspect.html
        # https://peps.python.org/pep-0362/
        sig = signature(function)
        sig_function = str(sig).replace(par_x_axis, "par_x_axis")
        function = function_name + str(sig_function)
        title = function_name + str(sig)
        title = title.replace("_", " ")

        sig = signature(correct_function)
        sig_function = str(sig).replace(par_x_axis, "par_x_axis")
        correct_function2 = str(correct_function.__name__) + sig_function

        # calculate the student's answer through eval()
        student_answer = [eval(function) for par_x_axis in horizontal_axis]

        # calculate the correct answer
        correct_answer = []
        for par_x_axis in horizontal_axis:
            correct_answer.append(eval(correct_function2))
        # This code should work, but it does not
        # correct_answer = [eval(correct_function2) for par_x_axis in horizontal_axis]

        # check if the answer is correct and plot it before/below lines
        changes = np.array(student_answer) - np.array(correct_answer)
        student_answer = [1e-99 if x == 0 else x for x in student_answer]
        inaccuracy = np.abs(1 - np.array(correct_answer) / np.array(student_answer))

        # Check if the answer is correct and provide textual feedback
        label = "Inaccurate"
        y_loc = (np.mean(correct_answer) + np.min(correct_answer)) / 2
        if np.max(changes) == 0:
            label = "Perfect!"

        if np.max(changes) != 0 and np.max(inaccuracy) < f_margin:
            label = "Good!"
        if new_graph == True and label != "Inaccurate":
            text = ax.text(
                np.mean([horizontal_axis]),
                y_loc,
                label,
                fontsize=14,
                color="#1b5a00",
                ha="center",
                va="center",
            )

        # plot the answers
        if new_graph == True:
            line = ax.plot(horizontal_axis, student_answer, label="Your answer")
            line = ax.plot(horizontal_axis, correct_answer, label="Correct answer")
        else:
            function_name_spaces = function_name.replace("_", " ")
            line = ax.plot(
                horizontal_axis,
                student_answer,
                label=function_name_spaces + " (" + label + ")",
                alpha=0.75,
            )

        # set the legend
        if new_graph == True:
            ax.legend(loc="best")
        else:
            ax.legend(bbox_to_anchor=(-0.02, 1), loc="lower left")

        # Set title and show the result if this is a new figure
        if new_graph == True:
            ax.set_title(title)
            display(pane)  # update the graph

    except:  # if the graph can not be plotted
        if function_name in globals():
            if new_graph == True:
                text_failed = (
                    "Almost there, \n your function \n"
                    + str(function_name)
                    + "\n can not be plotted, \n please try to fix the bug."
                )
                x_ticks = ax.get_xticks()
                y_ticks = ax.get_yticks()
                # text = ax.text(np.average(x_ticks),np.average(y_ticks), text_failed, fontsize=12, color = 'r', ha='center', va='center')
                text = fig.text(
                    0.5,
                    0.5,
                    text_failed,
                    fontsize=12,
                    color="r",
                    ha="center",
                    va="center",
                )
                display(pane)  # update the graph
            else:
                print(
                    "Almost there, your function "
                    + str(function_name)
                    + " can not be plotted, please try to fix the bug."
                )

        else:
            print("Careful! The function " + function_name + " is not defined.")

    return pane, ax

In [ ]:
def nummerical_subquestions(
    Questions,
    answer_question,
    label_question=None,
    Unit_question=None,
    FB_G_question=None,
    FB_W_question=None,
):
    if label_question == None:
        label_question = [[""] * len(Questions)] * len(answer_question)

    if Unit_question == None:
        Unit_question = [""] * len(Questions)

    if FB_G_question == None:
        FB_G_question = [""] * len(Questions)

    if FB_W_question == None:
        FB_W_question = [""] * len(Questions)

    FV = classify_variables(locals())
    output = nummeric_subquestion_body(FV)
    return output


def check_answers_nummeric_subquestion(AV):
    def button_callback(b):
        AV.attempt.value += 1
        score = 0
        # print('Debug: id:', FV.id, ' value: ', FV.attempt.value)

        responses = []
        for i in range(len(AV.answers)):
            num_widget = AV.num_widgets[i]
            feedback_widget = AV.feedback_widgets[i]

            answer = AV.answers[i]
            response = AV.num_widgets[i].value

            # if answer is correct
            if np.abs(answer - num_widget.value) < limit_answer(answer):
                score += 1
                if len(AV.FB_G) != 0:
                    feedback_widget.value = AV.FB_G[i]
                else:
                    feedback_widget.value = "Well done, this is correct!"

            # the answer is NOT within boundaries, provide feedback based on the number of attempts
            if np.abs(answer - num_widget.value) >= limit_answer(answer):
                if AV.attempt.value < 3 and len(AV.FB_W) > 0:
                    feedback_widget.value = FB_W[i]

                if AV.attempt.value < 3 and len(AV.FB_W) == 0:
                    feedback_widget.value = "Oops, there seems to be a mistake"

                if AV.attempt.value >= 3:
                    feedback_widget.value = (
                        "The correct answer is "
                        + str(answer)
                        + " "
                        + str(AV.unit)
                        + "."
                    )

        AV.final_score_widget.value = str(score) + "/" + str(len(AV.answers))

    return button_callback  # otherwise returns TypeError: 'NoneType' object is not callable


def nummeric_subquestion_body(FV):
    all_question_widgets = []
    all_answers = []
    attempts = (
        []
    )  # widgets only used for counting how many times the submit button is pressed
    id = 0
    for i, (question, unit, answers, labels, FB_G, FB_W) in enumerate(
        zip(
            FV.Questions,
            FV.Unit_question,
            FV.answer_question,
            FV.label_question,
            FV.FB_G_question,
            FV.FB_W_question,
        )
    ):
        id += 1
        question_widget = pn.widgets.StaticText(value=question, width=750)

        # Add the subquestions, which come underneath each other
        Rows_answer = []
        num_widgets = []
        feedback_widgets = []
        for number, answer in enumerate(answers):
            # set subquestion/label on the left side
            if len(labels) > number:
                label = labels[number]
            else:
                label = str(number + 1) + str(")")

            max_length = max(len(label) for label in labels)

            # make the widgets
            number_widget = pn.widgets.StaticText(
                value=str(label), width=max_length * 8
            )
            unit_widget = pn.widgets.StaticText(value=unit, width=50)
            num_widget = pn.widgets.FloatInput(value=0, step=0.01, width=100)
            feedback_widget = pn.widgets.StaticText(value="", name="")  # , width=250)

            num_widgets.append(num_widget)
            feedback_widgets.append(feedback_widget)

            Hbox = pn.Row(number_widget, num_widget, unit_widget, feedback_widget)
            Rows_answer.append(Hbox)
        all_answers.append(num_widgets)

        # Add a submit button with a widget that returns the final score
        submit_button = pn.widgets.Button(name="Check")
        submit_text_widget = pn.widgets.StaticText(value="Final score:", width=70)
        final_score_widget = pn.widgets.TextInput(value="", name="", width=130)
        row_submit = pn.Row(submit_button, submit_text_widget, final_score_widget)

        # Build the UI (User Interface)
        question_widget = pn.Column(question_widget, *Rows_answer, row_submit)

        # Add functionality to check the number of attempts for each subquestion
        # The values for the submit button are determined at the moment these are created.
        attempt = pn.widgets.FloatInput(value=0)
        attempts.append(attempt)
        all_question_widgets.append(question_widget)

        # Store the variable for the question, and make the submit button work.
        FV2 = classify_variables(locals())
        print(FV2.FB_W)
        submit_button.on_click(check_answers_nummeric_subquestion(FV2))

    return pn.Column(*all_question_widgets)